In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

import sqlite3
from datetime import datetime
from tqdm import tqdm
import locale

def scrap_amf():
    url_base = "https://www.amf-france.org/fr/actualites-publications/la-une/toutes-les-actualites-et-publications"
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)

    driver.get(url_base)
    # Réglage de la locale pour interpréter correctement les noms de mois en français
    locale.setlocale(locale.LC_TIME, 'fr_FR')
    conn = sqlite3.connect('new_amf.db')
    cursor = conn.cursor()

    cursor.execute('''
            CREATE TABLE IF NOT EXISTS amf_articles (
                id INTEGER PRIMARY KEY,
                title TEXT NOT NULL,
                date DATE,
                link TEXT NOT NULL,
                source TEXT NOT NULL,
                content TEXT
            )
        ''')
    conn.commit()
    # Attendre que la table soit visible
    full_xpath = "/html/body/div[2]/div[1]/main/div[3]/div/div[2]/div/div[2]/div/div[2]/div/div[2]/table"
    WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, full_xpath)))

    try:
        # Boucle pour naviguer à travers les pages
        while True:
            # Récupérer le numéro de la page courante
            current_page = driver.find_element(By.CLASS_NAME, 'paginate_button.current').text
            print(f"Page actuelle : {current_page}")

            # Insérer ici votre logique pour extraire les données de la page actuelle
            try:
                rows = driver.find_elements(By.XPATH, f"{full_xpath}/tbody/tr")
                urls = [row.find_element(By.XPATH, ".//td[3]/a").get_attribute('href') for row in rows]

                # Traiter chaque URL
                for url in tqdm(urls, desc="Traitement des articles"):
                    # Ouvrir un nouvel onglet pour chaque URL
                    driver.execute_script("window.open();")
                    driver.switch_to.window(driver.window_handles[1])  # Passer à l'onglet nouvellement ouvert
                    driver.get(url)
                    #print(f"Traitement de l'URL : {url}")
                    
                    article = {'URL': url}

                    try:
                        article['Titre'] = driver.find_element(By.CSS_SELECTOR, '.like-h1').text
                    except:
                        article['Titre'] = ""

                    try:
                        article['Date'] = driver.find_element(By.CSS_SELECTOR, '.date').text
                        raw_date = datetime.strptime(article['Date'], "%d %B %Y")
                        # Vérifier si la date est antérieure à 2023
                        if raw_date.year < 2023:
                            print("Date antérieure à 2023. Arrêt du scraping.")
                            return
                        article['Date'] = raw_date.strftime("%Y-%m-%d")  # Format ISO pour stockage
                    except ValueError:
                        print(f"Format de date non reconnu : {raw_date}")
                        continue
                    
                    try:
                        content_divs = driver.find_elements(By.CSS_SELECTOR, 
                                                                "div.paragraph.paragraph--type--wysiwyg.paragraph--view-mode--default, div.intro")
                        article['Contenu'] = ' '.join([div.text for div in content_divs])
                    except:
                        article['Contenu'] = ""

                    # Insertion dans la base de données
                    if all(field != "" for field in article.values()):
                        try:
                            cursor.execute('SELECT COUNT(*) FROM amf_articles WHERE link = ?', (article['URL'],))
                            if cursor.fetchone()[0] == 0:
                                cursor.execute('INSERT INTO amf_articles (title, date, content, source, link) VALUES (?, ?, ?, ?, ?)', 
                                            (article['Titre'], article['Date'], article['Contenu'],"AMF", article['URL']))
                                conn.commit()
                                print(f"Titre: {article['Titre']}\nLien: {article['URL']}\nDate de publication: {article['Date']}\nContenu: {article['Contenu']}\n")
                            else :
                                print(f"Article déjà existant dans la base de données : {article['URL']}")
                                
                        except sqlite3.IntegrityError:
                            print(f"Erreur d'insertion dans la BDD : {article['URL']}")
                    # Fermer l'onglet actuel et retourner à l'onglet principal
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

            except NoSuchElementException:
                print("Impossible de trouver les lignes ou URLs sur la page principale.")
            
            try :    
                # Passer à la page suivante en utilisant JavaScript pour cliquer
                next_button = driver.find_element(By.ID, 'DataTables_Table_0_next')
                if 'disabled' not in next_button.get_attribute('class'):
                    driver.execute_script("arguments[0].click();", next_button)
                else:
                    print("next page impossible")
                    break  # Sortir de la boucle si le bouton Suivant est désactivé (dernière page)
            except :
                print("erreur bouton next page")

            # Attente pour assurer le chargement de la page suivante
            WebDriverWait(driver, 30).until(
                EC.staleness_of(next_button)
            )

    except : 
        print("Erreur lors du scraping.")
    finally:
        driver.quit()
        conn.close()

In [21]:
scrap_amf()

Page actuelle : 1


Traitement des articles:  10%|█         | 1/10 [00:00<00:03,  2.47it/s]

Article déjà existant dans la base de données : https://www.amf-france.org/fr/actualites-publications/actualites/cybersecurite-et-risques-informatiques-lamf-appelle-les-acteurs-se-preparer-lentree-en-application


Traitement des articles:  20%|██        | 2/10 [00:00<00:03,  2.67it/s]

Article déjà existant dans la base de données : https://www.amf-france.org/fr/actualites-publications/actualites/lamf-rappelle-les-regles-didentification-des-investisseurs-particuliers-dans-le-reporting-des


...

Article déjà existant dans la base de données : https://www.amf-france.org/fr/actualites-publications/evenements-de-lamf/colloques-et-conferences-de-lamf/webinaire-rapport-amf-sur-le-reporting-taxonomie-des-societes-cotees-non-financieres-video


Traitement des articles:  30%|███       | 3/10 [00:02<00:05,  1.23it/s]

Article déjà existant dans la base de données : https://www.amf-france.org/fr/actualites-publications/evenements-de-lamf/colloques-et-conferences-de-lamf/le-conseil-en-investissement-et-les-exigences-en-matiere-dadequation-atelier-amf


Traitement des articles:  30%|███       | 3/10 [00:02<00:05,  1.27it/s]


Erreur lors du scraping.
